In [28]:
import random
import sys
import numpy as np
import pandas as pd
from sklearn.preprocessing import (StandardScaler, MinMaxScaler)
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (mean_absolute_error, mean_squared_error,)
from xgboost import XGBRegressor

In [29]:
df = pd.read_excel("NFL Champion RawData.xlsx")
df.head(2)

,Season,Team ID,Team ID/Season,Team,Team/Season,sum mvp share,sum dpoy share,sum opoy share,all pro,cpoy share,...,team_defense_Rushing_Yds,team_defense_Rushing_TD,team_defense_Rushing_Y/A,team_defense_Rushing_1stD,team_defense_Penalties_Pen,team_defense_Penalties_Yds,team_defense_Penalties_1stPy,team_defense_Sc%,team_defense_TO%,team_defense_EXP
0,2022,1,1 2022,Kansas City Chiefs,Kansas City Chiefs 2022,0.86,0.0,0.78,9,0.0,...,1823,10,4.4,102,98,734,28,35.9,10.3,-65.59
1,2022,17,17 2022,Philadelphia Eagles,Philadelphia Eagles 2022,0.00,0.0,0.00,22,0.0,...,2068,15,4.6,115,95,798,24,32.4,15.1,20.73


In [30]:
co = df
print(list(co.columns))

['Season', 'Team ID', ' Team ID/Season', 'Team', 'Team/Season', 'sum mvp share', 'sum dpoy share', 'sum opoy share', 'all pro', 'cpoy share', 'sum playoff games', 'sum champion', 'L1Y cs', 'L3Y cs', 'L5Y cs', 'L8Y cs', 'W', 'L', 'T', 'W-L%', 'PF', 'PA', 'PD', 'MoV', 'SoS', 'SRS', 'OSRS', 'DSRS', 'make playoffs', 'rk playoffs', '1 seed', '2 seed', '3 seed', '4 seed', '5 seed', '6 seed', 'champion share', 'pso', 'team_offense_G', 'team_offense_PF', 'team_offense_Tot Yds & TO_Yds', 'team_offense_Tot Yds & TO_Ply', 'team_offense_Tot Yds & TO_Y/P', 'team_offense_Tot Yds & TO_TO', 'team_offense_Tot Yds & TO_FL', 'team_offense_Tot Yds & TO_1stD', 'team_offense_Tot Yds & TO_Cmp', 'team_offense_Tot Yds & TO_Att', 'team_offense_Passing_Yds', 'team_offense_Passing_TD', 'team_offense_Passing_Int', 'team_offense_Passing_NY/A', 'team_offense_Passing_1stD', 'team_offense_Passing_Att', 'team_offense_Rushing_Yds', 'team_offense_Rushing_TD', 'team_offense_Rushing_Y/A', 'team_offense_Rushing_1stD', 'team

In [31]:
#Dropping all data before 2000, as this dataset begins in 1990, so stats don't fully come into focus until 2000s.

df.drop(df[df['Season'] < 2000].index, inplace = True)
df

,Season,Team ID,Team ID/Season,Team,Team/Season,sum mvp share,sum dpoy share,sum opoy share,all pro,cpoy share,...,team_defense_Rushing_Yds,team_defense_Rushing_TD,team_defense_Rushing_Y/A,team_defense_Rushing_1stD,team_defense_Penalties_Pen,team_defense_Penalties_Yds,team_defense_Penalties_1stPy,team_defense_Sc%,team_defense_TO%,team_defense_EXP
0,2022,1,1 2022,Kansas City Chiefs,Kansas City Chiefs 2022,0.860,0.000,0.780,9,0.00,...,1823,10,4.4,102,98,734,28,35.9,10.3,-65.59
1,2022,17,17 2022,Philadelphia Eagles,Philadelphia Eagles 2022,0.000,0.000,0.000,22,0.00,...,2068,15,4.6,115,95,798,24,32.4,15.1,20.73
2,2022,19,19 2022,San Francisco 49ers,San Francisco 49ers 2022,0.000,0.000,0.240,10,0.30,...,1321,11,3.4,77,96,832,26,25.7,15.3,56.26
3,2022,2,2 2022,Buffalo Bills,Buffalo Bills 2022,0.080,0.380,0.060,16,0.16,...,1673,10,4.3,103,89,754,29,31.0,14.3,4.46
4,2022,18,18 2022,Minnesota Vikings,Minnesota Vikings 2022,0.000,0.020,0.000,21,0.02,...,2093,18,4.5,113,111,926,22,41.1,12.0,-104.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,2000,3,3 2000,Cincinnati Bengals,Cincinnati Bengals 2000,0.000,0.000,0.000,2,0.00,...,1925,12,3.8,101,115,924,24,32.8,9.7,136.24
730,2000,29,29 2000,Atlanta Falcons,Atlanta Falcons 2000,0.085,0.000,0.043,7,0.00,...,1983,16,4.4,113,122,1010,20,37.0,12.5,88.81
731,2000,11,11 2000,Cleveland Browns,Cleveland Browns 2000,0.000,0.000,0.000,0,0.00,...,2505,26,4.2,147,71,626,27,39.1,10.9,49.34
732,2000,31,31 2000,Arizona Cardinals,Arizona Cardinals 2000,0.000,0.000,0.000,6,0.00,...,2609,29,4.5,149,96,914,21,42.1,11.2,25.06


Model Workings:
    
1st Approach: Past Testing, predicting the last 10 nfl champions as accurately as possible. (2022-2013)

2nd Approach: Current Testing, training all data to year in which prediction is needed.

In [32]:
co = df
print(list(co.columns))

['Season', 'Team ID', ' Team ID/Season', 'Team', 'Team/Season', 'sum mvp share', 'sum dpoy share', 'sum opoy share', 'all pro', 'cpoy share', 'sum playoff games', 'sum champion', 'L1Y cs', 'L3Y cs', 'L5Y cs', 'L8Y cs', 'W', 'L', 'T', 'W-L%', 'PF', 'PA', 'PD', 'MoV', 'SoS', 'SRS', 'OSRS', 'DSRS', 'make playoffs', 'rk playoffs', '1 seed', '2 seed', '3 seed', '4 seed', '5 seed', '6 seed', 'champion share', 'pso', 'team_offense_G', 'team_offense_PF', 'team_offense_Tot Yds & TO_Yds', 'team_offense_Tot Yds & TO_Ply', 'team_offense_Tot Yds & TO_Y/P', 'team_offense_Tot Yds & TO_TO', 'team_offense_Tot Yds & TO_FL', 'team_offense_Tot Yds & TO_1stD', 'team_offense_Tot Yds & TO_Cmp', 'team_offense_Tot Yds & TO_Att', 'team_offense_Passing_Yds', 'team_offense_Passing_TD', 'team_offense_Passing_Int', 'team_offense_Passing_NY/A', 'team_offense_Passing_1stD', 'team_offense_Passing_Att', 'team_offense_Rushing_Yds', 'team_offense_Rushing_TD', 'team_offense_Rushing_Y/A', 'team_offense_Rushing_1stD', 'team

In [33]:
#RandomForest Regressor
#PastApproach

sc = MinMaxScaler()
features = ['SRS','1 seed', '2 seed', '3 seed', '4 seed', '5 seed', '6 seed','sum mvp share', 'sum dpoy share', 
            'sum opoy share', 'all pro', 'cpoy share', 'sum playoff games', 'sum champion', 'L1Y cs', 'L3Y cs',
            'L5Y cs', 'L8Y cs','PF', 'PA', 'SoS', 'OSRS', 'DSRS',   'pso', 'team_offense_G',
            'team_offense_PF', 'team_offense_Tot Yds & TO_Yds', 'team_offense_Tot Yds & TO_Ply', 'team_offense_Tot Yds & TO_Y/P', 
            'team_offense_Tot Yds & TO_TO', 'team_offense_Tot Yds & TO_FL', 'team_offense_Tot Yds & TO_1stD', 'team_offense_Tot Yds & TO_Cmp',
            'team_offense_Tot Yds & TO_Att', 'team_offense_Passing_Yds', 'team_offense_Passing_TD', 'team_offense_Passing_Int', 'team_offense_Passing_NY/A', 
            'team_offense_Passing_1stD', 'team_offense_Passing_Att', 'team_offense_Rushing_Yds', 'team_offense_Rushing_TD', 'team_offense_Rushing_Y/A', 
            'team_offense_Rushing_1stD', 'team_offense_Penalties_Pen', 'team_offense_Penalties_Yds', 'team_offense_Penalties_1stPy', 'team_offense_Sc%', 
            'team_offense_TO%', 'team_offense_EXP', 'team_defense_G', 'team_defense_PA', 'team_defense_Tot Yds & TO_Yds', 'team_defense_Tot Yds & TO_Ply', 
            'team_defense_Tot Yds & TO_Y/P', 'team_defense_Tot Yds & TO_TO', 'team_defense_Tot Yds & TO_FL', 'team_defense_Tot Yds & TO_1stD', 
            'team_defense_Tot Yds & TO_Cmp', 'team_defense_Tot Yds & TO_Att', 'team_defense_Passing_Yds', 'team_defense_Passing_TD', 
            'team_defense_Passing_Int', 'team_defense_Passing_NY/A', 'team_defense_Passing_1stD', 'team_defense_Passing_Att', 'team_defense_Rushing_Yds', 
            'team_defense_Rushing_TD', 'team_defense_Rushing_Y/A', 'team_defense_Rushing_1stD', 'team_defense_Penalties_Pen', 'team_defense_Penalties_Yds', 
            'team_defense_Penalties_1stPy', 'team_defense_Sc%', 'team_defense_TO%', 'team_defense_EXP']

train = df[df["Season"] < 2012].copy()
test = df[df["Season"] > 2012].copy()

#scaling
sc.fit(train[features])
train[features]=sc.transform(train[features])
test[features]=sc.transform(test[features])
rf = RandomForestRegressor(n_estimators=500, random_state=50, min_samples_split=2, max_depth=5)
rf.fit(train[features], train["champion share"])

#predicting
predictions = rf.predict(test[features]) 
predictions = pd.DataFrame(predictions, columns=["predicted champion share"], index=test.index)
frame = pd.concat([test[["Season", "Team", "champion share"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])
#sc.transform(df[features])

#result preview
print("Results:")
print("=============================================================================\n")
print(frame.head(5))
print("-----------------------------------------------------------------------------\n")

#error metrics
mae = mean_absolute_error(frame["champion share"], frame["predicted champion share"])
print("Error:")
print("=============================================================================\n")
print("MAE: ", mae)
print("-----------------------------------------------------------------------------\n")

#feature importance
f_list = list(features)
f_importance = pd.Series(rf.feature_importances_, index = f_list).sort_values(ascending = False)
print("Feature Importance")
print("=============================================================================\n")
print(f_importance.head(10))
print("-----------------------------------------------------------------------------\n")

#results in full
print("Full Results:")
print("=============================================================================\n")
predictions = rf.predict(test[features]) 
predictions = pd.DataFrame(predictions*100, columns=["predicted champion share"], index=test.index)
frame = pd.concat([test[["Season", "Team", "champion share"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])

#print(frame.to_string())




Results:

   Season                 Team  champion share  predicted champion share
0    2022   Kansas City Chiefs            1.00                  0.512296
3    2022        Buffalo Bills            0.25                  0.413830
1    2022  Philadelphia Eagles            0.75                  0.397455
2    2022  San Francisco 49ers            0.50                  0.386103
5    2022   Cincinnati Bengals            0.50                  0.248893
-----------------------------------------------------------------------------

Error:

MAE:  0.09606652201114668
-----------------------------------------------------------------------------

Feature Importance
SRS                         0.336020
2 seed                      0.083910
1 seed                      0.060295
PA                          0.044102
team_defense_PA             0.037668
team_offense_Passing_Yds    0.031017
team_defense_EXP            0.030649
DSRS                        0.025018
4 seed                      0.016205
team_def

In [35]:
#Ridge Regressor
#PastApproach

sc = MinMaxScaler()
features = ['SRS','1 seed', '2 seed', '3 seed', '4 seed', '5 seed', '6 seed','sum mvp share', 'sum dpoy share', 
            'sum opoy share', 'all pro', 'cpoy share', 'sum playoff games', 'sum champion', 'L1Y cs', 'L3Y cs',
            'L5Y cs', 'L8Y cs','PF', 'PA', 'SoS', 'OSRS', 'DSRS',   'pso', 'team_offense_G',
            'team_offense_PF', 'team_offense_Tot Yds & TO_Yds', 'team_offense_Tot Yds & TO_Ply', 'team_offense_Tot Yds & TO_Y/P', 
            'team_offense_Tot Yds & TO_TO', 'team_offense_Tot Yds & TO_FL', 'team_offense_Tot Yds & TO_1stD', 'team_offense_Tot Yds & TO_Cmp',
            'team_offense_Tot Yds & TO_Att', 'team_offense_Passing_Yds', 'team_offense_Passing_TD', 'team_offense_Passing_Int', 'team_offense_Passing_NY/A', 
            'team_offense_Passing_1stD', 'team_offense_Passing_Att', 'team_offense_Rushing_Yds', 'team_offense_Rushing_TD', 'team_offense_Rushing_Y/A', 
            'team_offense_Rushing_1stD', 'team_offense_Penalties_Pen', 'team_offense_Penalties_Yds', 'team_offense_Penalties_1stPy', 'team_offense_Sc%', 
            'team_offense_TO%', 'team_offense_EXP', 'team_defense_G', 'team_defense_PA', 'team_defense_Tot Yds & TO_Yds', 'team_defense_Tot Yds & TO_Ply', 
            'team_defense_Tot Yds & TO_Y/P', 'team_defense_Tot Yds & TO_TO', 'team_defense_Tot Yds & TO_FL', 'team_defense_Tot Yds & TO_1stD', 
            'team_defense_Tot Yds & TO_Cmp', 'team_defense_Tot Yds & TO_Att', 'team_defense_Passing_Yds', 'team_defense_Passing_TD', 
            'team_defense_Passing_Int', 'team_defense_Passing_NY/A', 'team_defense_Passing_1stD', 'team_defense_Passing_Att', 'team_defense_Rushing_Yds', 
            'team_defense_Rushing_TD', 'team_defense_Rushing_Y/A', 'team_defense_Rushing_1stD', 'team_defense_Penalties_Pen', 'team_defense_Penalties_Yds', 
            'team_defense_Penalties_1stPy', 'team_defense_Sc%', 'team_defense_TO%', 'team_defense_EXP']

train = df[df["Season"] < 2012].copy()
test = df[df["Season"] > 2012].copy()

#scaling
sc.fit(train[features])
train[features]=sc.transform(train[features])
test[features]=sc.transform(test[features])
reg = Ridge(alpha=.1)
reg.fit(train[features], train['champion share'])

#predicting
predictions = reg.predict(test[features]) 
predictions = pd.DataFrame(predictions, columns=["predicted champion share"], index=test.index)
frame = pd.concat([test[["Season", "Team", "champion share"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])
#sc.transform(df[features])

#result preview
print("Results:")
print("=============================================================================\n")
print(frame.head(5))
print("-----------------------------------------------------------------------------\n")

#error metrics
mae = mean_absolute_error(frame["champion share"], frame["predicted champion share"])
print("Error:")
print("=============================================================================\n")
print("MAE: ", mae)
print("-----------------------------------------------------------------------------\n")

#feature importance
f_list = list(features)
f_importance = pd.Series(reg.coef_, index = f_list).sort_values(ascending = False)
print("Feature Importance")
print("=============================================================================\n")
print(f_importance.head(10))
print("-----------------------------------------------------------------------------\n")

#results in full
print("Full Results:")
print("=============================================================================\n")
predictions = reg.predict(test[features]) 
predictions = pd.DataFrame(predictions*100, columns=["predicted champion share"], index=test.index)
frame = pd.concat([test[["Season", "Team", "champion share"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])

#print(frame.to_string())



Results:

   Season                  Team  champion share  predicted champion share
3    2022         Buffalo Bills            0.25                  0.548562
1    2022   Philadelphia Eagles            0.75                  0.540193
2    2022   San Francisco 49ers            0.50                  0.455022
0    2022    Kansas City Chiefs            1.00                  0.411059
9    2022  Jacksonville Jaguars            0.25                  0.167330
-----------------------------------------------------------------------------

Error:

MAE:  0.1363676363168397
-----------------------------------------------------------------------------

Feature Importance

team_defense_EXP                 0.552821
1 seed                           0.513644
sum playoff games                0.507917
2 seed                           0.495016
team_offense_TO%                 0.392049
team_defense_Rushing_Yds         0.375131
team_defense_Tot Yds & TO_Yds    0.278568
L5Y cs                           0.264934

In [36]:
#XG Boost Regressor
#PastApproach

sc = MinMaxScaler()
features = ['SRS','1 seed', '2 seed', '3 seed', '4 seed', '5 seed', '6 seed','sum mvp share', 'sum dpoy share', 
            'sum opoy share', 'all pro', 'cpoy share', 'sum playoff games', 'sum champion', 'L1Y cs', 'L3Y cs',
            'L5Y cs', 'L8Y cs','PF', 'PA', 'SoS', 'OSRS', 'DSRS',   'pso', 'team_offense_G',
            'team_offense_PF', 'team_offense_Tot Yds & TO_Yds', 'team_offense_Tot Yds & TO_Ply', 'team_offense_Tot Yds & TO_Y/P', 
            'team_offense_Tot Yds & TO_TO', 'team_offense_Tot Yds & TO_FL', 'team_offense_Tot Yds & TO_1stD', 'team_offense_Tot Yds & TO_Cmp',
            'team_offense_Tot Yds & TO_Att', 'team_offense_Passing_Yds', 'team_offense_Passing_TD', 'team_offense_Passing_Int', 'team_offense_Passing_NY/A', 
            'team_offense_Passing_1stD', 'team_offense_Passing_Att', 'team_offense_Rushing_Yds', 'team_offense_Rushing_TD', 'team_offense_Rushing_Y/A', 
            'team_offense_Rushing_1stD', 'team_offense_Penalties_Pen', 'team_offense_Penalties_Yds', 'team_offense_Penalties_1stPy', 'team_offense_Sc%', 
            'team_offense_TO%', 'team_offense_EXP', 'team_defense_G', 'team_defense_PA', 'team_defense_Tot Yds & TO_Yds', 'team_defense_Tot Yds & TO_Ply', 
            'team_defense_Tot Yds & TO_Y/P', 'team_defense_Tot Yds & TO_TO', 'team_defense_Tot Yds & TO_FL', 'team_defense_Tot Yds & TO_1stD', 
            'team_defense_Tot Yds & TO_Cmp', 'team_defense_Tot Yds & TO_Att', 'team_defense_Passing_Yds', 'team_defense_Passing_TD', 
            'team_defense_Passing_Int', 'team_defense_Passing_NY/A', 'team_defense_Passing_1stD', 'team_defense_Passing_Att', 'team_defense_Rushing_Yds', 
            'team_defense_Rushing_TD', 'team_defense_Rushing_Y/A', 'team_defense_Rushing_1stD', 'team_defense_Penalties_Pen', 'team_defense_Penalties_Yds', 
            'team_defense_Penalties_1stPy', 'team_defense_Sc%', 'team_defense_TO%', 'team_defense_EXP']

train = df[df["Season"] < 2012].copy()
test = df[df["Season"] > 2012].copy()

#scaling
sc.fit(train[features])
train[features]=sc.transform(train[features])
test[features]=sc.transform(test[features])
xgb = XGBRegressor(n_estimators=500,learning_rate=.05)
xgb.fit(train[features], train['champion share'])

#predicting
predictions = xgb.predict(test[features]) 
predictions = pd.DataFrame(predictions, columns=["predicted champion share"], index=test.index)
frame = pd.concat([test[["Season", "Team", "champion share"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])
#sc.transform(df[features])

#result preview
print("Results:")
print("=============================================================================\n")
print(frame.head(5))
print("-----------------------------------------------------------------------------\n")

#error metrics
mae = mean_absolute_error(frame["champion share"], frame["predicted champion share"])
print("Error:")
print("=============================================================================\n")
print("MAE: ", mae)
print("-----------------------------------------------------------------------------\n")

#feature importance
f_list = list(features)
f_importance = pd.Series(xgb.feature_importances_, index = f_list).sort_values(ascending = False)
print("Feature Importance")
print("=============================================================================\n")
print(f_importance.head(10))
print("-----------------------------------------------------------------------------\n")

#results in full
print("Full Results:")
print("=============================================================================\n")
predictions = xgb.predict(test[features]) 
predictions = pd.DataFrame(predictions*100, columns=["predicted champion share"], index=test.index)
frame = pd.concat([test[["Season", "Team", "champion share"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])

#print(frame.to_string())

Results:

    Season                  Team  champion share  predicted champion share
0     2022    Kansas City Chiefs            1.00                  0.556037
18    2022  Tampa Bay Buccaneers            0.00                  0.457922
1     2022   Philadelphia Eagles            0.75                  0.433863
2     2022   San Francisco 49ers            0.50                  0.412527
9     2022  Jacksonville Jaguars            0.25                  0.392775
-----------------------------------------------------------------------------

Error:

MAE:  0.09775478211436166
-----------------------------------------------------------------------------

Feature Importance

1 seed                           0.125051
PA                               0.096927
2 seed                           0.090434
team_offense_Tot Yds & TO_Cmp    0.058052
SRS                              0.055016
DSRS                             0.048253
team_offense_Tot Yds & TO_Yds    0.038353
team_offense_Passing_1stD        0

Present Test: Using RandomForest 

In [44]:
#RandomForest Regressor
#PresentTest Approach

sc = MinMaxScaler()
features = ['1 seed', '2 seed', '3 seed', '4 seed', '5 seed', '6 seed','sum mvp share', 'sum dpoy share', 
            'sum opoy share', 'all pro', 'cpoy share', 'sum playoff games', 'sum champion', 'L1Y cs', 'L3Y cs',
            'L5Y cs', 'L8Y cs','PF', 'PA','SoS', 'SRS', 'OSRS', 'DSRS',   'pso', 'team_offense_G',
            'team_offense_PF', 'team_offense_Tot Yds & TO_Yds', 'team_offense_Tot Yds & TO_Ply', 'team_offense_Tot Yds & TO_Y/P', 
            'team_offense_Tot Yds & TO_TO', 'team_offense_Tot Yds & TO_FL', 'team_offense_Tot Yds & TO_1stD', 'team_offense_Tot Yds & TO_Cmp',
            'team_offense_Tot Yds & TO_Att', 'team_offense_Passing_Yds', 'team_offense_Passing_TD', 'team_offense_Passing_Int', 'team_offense_Passing_NY/A', 
            'team_offense_Passing_1stD', 'team_offense_Passing_Att', 'team_offense_Rushing_Yds', 'team_offense_Rushing_TD', 'team_offense_Rushing_Y/A', 
            'team_offense_Rushing_1stD', 'team_offense_Penalties_Pen', 'team_offense_Penalties_Yds', 'team_offense_Penalties_1stPy', 'team_offense_Sc%', 
            'team_offense_TO%', 'team_offense_EXP', 'team_defense_G', 'team_defense_PA', 'team_defense_Tot Yds & TO_Yds', 'team_defense_Tot Yds & TO_Ply', 
            'team_defense_Tot Yds & TO_Y/P', 'team_defense_Tot Yds & TO_TO', 'team_defense_Tot Yds & TO_FL', 'team_defense_Tot Yds & TO_1stD', 
            'team_defense_Tot Yds & TO_Cmp', 'team_defense_Tot Yds & TO_Att', 'team_defense_Passing_Yds', 'team_defense_Passing_TD', 
            'team_defense_Passing_Int', 'team_defense_Passing_NY/A', 'team_defense_Passing_1stD', 'team_defense_Passing_Att', 'team_defense_Rushing_Yds', 
            'team_defense_Rushing_TD', 'team_defense_Rushing_Y/A', 'team_defense_Rushing_1stD', 'team_defense_Penalties_Pen', 'team_defense_Penalties_Yds', 
            'team_defense_Penalties_1stPy', 'team_defense_Sc%', 'team_defense_TO%', 'team_defense_EXP']

train = df[df["Season"] < 2021].copy()
test = df[df["Season"] > 2021].copy()

#scaling
sc.fit(train[features])
train[features]=sc.transform(train[features])
test[features]=sc.transform(test[features])


rf = RandomForestRegressor(n_estimators=500, random_state=50, min_samples_split=2, max_depth=5)
rf.fit(train[features], train["champion share"])

predictions = rf.predict(test[features]) 
predictions = pd.DataFrame(predictions, columns=["predicted champion share"], index=test.index)
frame = pd.concat([test[["Season", "Team", "champion share"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])

#result preview
print("Result Preview:")
print("=============================================================================\n")
print(frame.head(5))
print("-----------------------------------------------------------------------------\n")

#error metrics
mae1 = mean_absolute_error(frame["champion share"], frame["predicted champion share"])
#Average Precision
#error metric by ranking, specifically the top 5 or what ranking range chosen, if that team is predicted in top 5 and actually finished top 5 it's 1.0, 100% success, but
#for instance if it took 6 ranks to find the 4th actual ranked entity then that would result in a valu of 4/6 so .6667
#so predicted 33th, but finished top 5 = 5/33 = .1515

#------------------------------------------------------------------------------------------
frame = pd.concat([test[["Team", "champion share"]], predictions], axis=1)
frame.sort_values(["champion share"], ascending=False).head(10)
actual = frame.sort_values("champion share", ascending=False)
predicted = frame.sort_values("predicted champion share", ascending=False)
actual["Rank"] = list(range(1,actual.shape[0]+1))
predicted["Predicted Rank"] = list(range(1,predicted.shape[0]+1))
actual.merge(predicted, on="Team").head(5)

def find_ap(frame):
    actual = frame.sort_values("champion share", ascending=False).head(5)
    predicted = frame.sort_values("predicted champion share", ascending=False)
    ps = []
    found = 0
    seen = 1
    for index,row in predicted.iterrows():
        if row["Team"] in actual["Team"].values:
            found += 1
            ps.append(found / seen)
        seen += 1
    return sum(ps) / len(ps)
ap1 = find_ap(frame)

print("Error Metrics: ")
print("=============================================================================\n")
print("PresentTest AP: ", ap1)
print("PresentTest MAE: ", mae1)
print("-----------------------------------------------------------------------------\n")
#------------------------------------------------------------------------------------------


#feature importance
f_list = list(features)
f_importance = pd.Series(rf.feature_importances_, index = f_list).sort_values(ascending = False)
print("Feature Importance: ")
print("=============================================================================\n")
print(f_importance.head(10))
print("-----------------------------------------------------------------------------\n")

#results in full
print("Full Results:")
print("=============================================================================\n")
predictions = rf.predict(test[features]) 
predictions = pd.DataFrame(predictions*100, columns=["predicted champion share"], index=test.index)
frame = pd.concat([test[["Season", "Team", "champion share"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])

print(frame.to_string())

Result Preview:

    Season                  Team  champion share  predicted champion share
0     2022    Kansas City Chiefs            1.00                  0.631677
1     2022   Philadelphia Eagles            0.75                  0.530585
2     2022   San Francisco 49ers            0.50                  0.482136
3     2022         Buffalo Bills            0.25                  0.467898
18    2022  Tampa Bay Buccaneers            0.00                  0.394524
-----------------------------------------------------------------------------

Error Metrics: 

PresentTest AP:  0.8761904761904761
PresentTest MAE:  0.07355579280087099
-----------------------------------------------------------------------------

Feature Importance: 

SRS                         0.453076
1 seed                      0.092828
2 seed                      0.090470
4 seed                      0.022109
DSRS                        0.018360
all pro                     0.016822
team_defense_EXP            0.015325
tea

In [53]:
#RandomForest Regressor
#PresentApproach
#BackTesting Results

#5 = 2005
#all data from 2000-2004 becomes training, and so on.

Seasons = list(range(2000,2023))

aps=[]
all_predictions=[]
for Season in Seasons[5:]:
    train=df[df["Season"]<Season]
    test=df[df["Season"]==Season]
    rf.fit(train[features], train['champion share'])
            
predictions = rf.predict(test[features]) 
predictions = pd.DataFrame(predictions, columns=["predicted champion share"], index=test.index)
frame = pd.concat([test[["Season", "Team", "champion share"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])       
all_predictions.append(frame)
aps.append(find_ap(frame))

ap2 = sum(aps)/len(aps)
mae2 = mean_absolute_error(frame["champion share"], frame["predicted champion share"])

print("Error Metrics: ")
print("=============================================================================\n")
print("BackTest AP: ", ap2)
print("BackTest MAE: ", mae2)
print("-----------------------------------------------------------------------------\n")

mae:  0.07348406416778958
Error Metrics: 

BackTest AP:  0.925
BackTest MAE:  0.07348406416778958
-----------------------------------------------------------------------------



In [54]:
#Ridge Regressor
#PresentTest Approach

sc = MinMaxScaler()
features = ['1 seed', '2 seed', '3 seed', '4 seed', '5 seed', '6 seed','sum mvp share', 'sum dpoy share', 
            'sum opoy share', 'all pro', 'cpoy share', 'sum playoff games', 'sum champion', 'L1Y cs', 'L3Y cs',
            'L5Y cs', 'L8Y cs','PF', 'PA','SoS', 'SRS', 'OSRS', 'DSRS',   'pso', 'team_offense_G',
            'team_offense_PF', 'team_offense_Tot Yds & TO_Yds', 'team_offense_Tot Yds & TO_Ply', 'team_offense_Tot Yds & TO_Y/P', 
            'team_offense_Tot Yds & TO_TO', 'team_offense_Tot Yds & TO_FL', 'team_offense_Tot Yds & TO_1stD', 'team_offense_Tot Yds & TO_Cmp',
            'team_offense_Tot Yds & TO_Att', 'team_offense_Passing_Yds', 'team_offense_Passing_TD', 'team_offense_Passing_Int', 'team_offense_Passing_NY/A', 
            'team_offense_Passing_1stD', 'team_offense_Passing_Att', 'team_offense_Rushing_Yds', 'team_offense_Rushing_TD', 'team_offense_Rushing_Y/A', 
            'team_offense_Rushing_1stD', 'team_offense_Penalties_Pen', 'team_offense_Penalties_Yds', 'team_offense_Penalties_1stPy', 'team_offense_Sc%', 
            'team_offense_TO%', 'team_offense_EXP', 'team_defense_G', 'team_defense_PA', 'team_defense_Tot Yds & TO_Yds', 'team_defense_Tot Yds & TO_Ply', 
            'team_defense_Tot Yds & TO_Y/P', 'team_defense_Tot Yds & TO_TO', 'team_defense_Tot Yds & TO_FL', 'team_defense_Tot Yds & TO_1stD', 
            'team_defense_Tot Yds & TO_Cmp', 'team_defense_Tot Yds & TO_Att', 'team_defense_Passing_Yds', 'team_defense_Passing_TD', 
            'team_defense_Passing_Int', 'team_defense_Passing_NY/A', 'team_defense_Passing_1stD', 'team_defense_Passing_Att', 'team_defense_Rushing_Yds', 
            'team_defense_Rushing_TD', 'team_defense_Rushing_Y/A', 'team_defense_Rushing_1stD', 'team_defense_Penalties_Pen', 'team_defense_Penalties_Yds', 
            'team_defense_Penalties_1stPy', 'team_defense_Sc%', 'team_defense_TO%', 'team_defense_EXP']

train = df[df["Season"] < 2021].copy()
test = df[df["Season"] > 2021].copy()

#scaling
sc.fit(train[features])
train[features]=sc.transform(train[features])
test[features]=sc.transform(test[features])

reg = Ridge(alpha=.1)
reg.fit(train[features], train['champion share'])

predictions = reg.predict(test[features]) 
predictions = pd.DataFrame(predictions, columns=["predicted champion share"], index=test.index)
frame = pd.concat([test[["Season", "Team", "champion share"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])

#result preview
print("Result Preview:")
print("=============================================================================\n")
print(frame.head(5))
print("-----------------------------------------------------------------------------\n")

#error metrics
mae1 = mean_absolute_error(frame["champion share"], frame["predicted champion share"])
#Average Precision
#error metric by ranking, specifically the top 5 or what ranking range chosen, if that team is predicted in top 5 and actually finished top 5 it's 1.0, 100% success, but
#for instance if it took 6 ranks to find the 4th actual ranked entity then that would result in a valu of 4/6 so .6667
#so predicted 33th, but finished top 5 = 5/33 = .1515

#------------------------------------------------------------------------------------------
frame = pd.concat([test[["Team", "champion share"]], predictions], axis=1)
frame.sort_values(["champion share"], ascending=False).head(10)
actual = frame.sort_values("champion share", ascending=False)
predicted = frame.sort_values("predicted champion share", ascending=False)
actual["Rank"] = list(range(1,actual.shape[0]+1))
predicted["Predicted Rank"] = list(range(1,predicted.shape[0]+1))
actual.merge(predicted, on="Team").head(5)

def find_ap(frame):
    actual = frame.sort_values("champion share", ascending=False).head(5)
    predicted = frame.sort_values("predicted champion share", ascending=False)
    ps = []
    found = 0
    seen = 1
    for index,row in predicted.iterrows():
        if row["Team"] in actual["Team"].values:
            found += 1
            ps.append(found / seen)
        seen += 1
    return sum(ps) / len(ps)
ap1 = find_ap(frame)

print("Error Metrics: ")
print("=============================================================================\n")
print("PresentTest AP: ", ap1)
print("PresentTest MAE: ", mae1)
print("-----------------------------------------------------------------------------\n")
#------------------------------------------------------------------------------------------


#feature importance
f_list = list(features)
f_importance = pd.Series(reg.coef_, index = f_list).sort_values(ascending = False)
print("Feature Importance: ")
print("=============================================================================\n")
print(f_importance.head(10))
print("-----------------------------------------------------------------------------\n")

#results in full
print("Full Results:")
print("=============================================================================\n")
predictions = reg.predict(test[features]) 
predictions = pd.DataFrame(predictions*100, columns=["predicted champion share"], index=test.index)
frame = pd.concat([test[["Season", "Team", "champion share"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])

print(frame.to_string())

Result Preview:

    Season                  Team  champion share  predicted champion share
0     2022    Kansas City Chiefs            1.00                  0.660032
1     2022   Philadelphia Eagles            0.75                  0.641519
3     2022         Buffalo Bills            0.25                  0.572958
2     2022   San Francisco 49ers            0.50                  0.518299
18    2022  Tampa Bay Buccaneers            0.00                  0.224211
-----------------------------------------------------------------------------

Error Metrics: 

PresentTest AP:  0.75
PresentTest MAE:  0.09235212376218349
-----------------------------------------------------------------------------

Feature Importance: 

1 seed                           0.582437
team_defense_EXP                 0.559385
2 seed                           0.488577
team_offense_EXP                 0.466159
L5Y cs                           0.329877
team_offense_TO%                 0.305995
team_defense_Rushing_Yds

In [55]:
#Ridge Regressor
#PresentApproach
#BackTesting Results

#5 = 2005
#all data from 2000-2004 becomes training, and so on.

Seasons = list(range(2000,2023))

aps=[]
all_predictions=[]
for Season in Seasons[5:]:
    train=df[df["Season"]<Season]
    test=df[df["Season"]==Season]
    reg.fit(train[features], train['champion share'])
            
predictions = reg.predict(test[features]) 
predictions = pd.DataFrame(predictions, columns=["predicted champion share"], index=test.index)
frame = pd.concat([test[["Season", "Team", "champion share"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])       
all_predictions.append(frame)
aps.append(find_ap(frame))

ap2 = sum(aps)/len(aps)
mae2 = mean_absolute_error(frame["champion share"], frame["predicted champion share"])
print("mae: ", mae2)

print("Error Metrics: ")
print("=============================================================================\n")
print("BackTest AP: ", ap2)
print("BackTest MAE: ", mae2)
print("-----------------------------------------------------------------------------\n")

mae:  0.09651593322206098
Error Metrics: 

BackTest AP:  0.7753968253968253
BackTest MAE:  0.09651593322206098
-----------------------------------------------------------------------------



In [56]:
#Export Prediction 
predictions = rf.predict(test[features]) 
predictions = pd.DataFrame(predictions, columns=["predicted champion share"], index=test.index)
combination = pd.concat([test[["Season", "Team",'Team/Season',
                               "champion share"]], predictions], axis=1)
combination.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])

combination.to_excel('NFL Champion Predict.xlsx')

#Export Importance
f_list = list(features)
f_importance = pd.Series(rf.feature_importances_, index = f_list).sort_values(ascending = False)
f_importance.head(10)

f_importance.to_excel('NFL Chp Fea Imp.xlsx')